#### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB (ДЗ)

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную, максимальную и валюту).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [63]:
from bs4 import BeautifulSoup as bs # для работы с HTML
import requests
import re
import pandas as pd
from pprint import pprint

Поиск QA, г. Санкт-Петербург

In [64]:
##https://hh.ru/search/vacancy?clusters=true&enable_snippets=true&selary=&st=searchVacancy&text=QA

In [65]:
main_link = 'https://spb.hh.ru'

In [67]:
prof = 'QA'

In [68]:
params = {'clusters':'true',\
          'enable_snippets':'true',\
          'selary':'', \
          'st': 'searchVacancy',\
          'text': prof,
          'page' : ' '
                  
         } #передадим параметры в словарик

In [69]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}

In [70]:
response = requests.get(main_link + '/search/vacancy/',params=params,headers=headers)

In [79]:
if response.status_code == 200:
    print(f' {response.status_code}')
else:
    dom = bs(response.text, 'html.parser')
    page_block = dom.find('div',{'data-qa': 'pager-block'})
    if not page_blok:
        last_page = 1
    else:
        last_page = int(page_block.find_all('a',{'class':'HH-Pager-Control'})[-2].getText())
        print(f'last_page {last_page}')
    
    for page in rage(0, last_page):
        params['page'] = page
        response = requests.get(main_link + '/search/vacancy/',params=paramas,headers=headers)
        dom = bs(response.text, 'html.parser')
        vacancy_list = dom.find_all('div', {'class': 'vacancy-serp-item'})
        if len(vacancy_list) == 0:
            print(f'Vacancies with QA "{prof}" ')
        else:       
            vacancies = []
            for vacancy in vacancy_list:
                vacancy_data = {}
        
                vacancy_name = vacancy.find('div', {'class': 'resume-search-item__name'}) \
                                      .getText() \
                                      .replace(u'/xa0', u' ')
                vacancy_data['vacancy_name'] = vacancy_name
            
                compay_name = vacancy.find('div', {'class' : 'vacancy-serp-item__meta-info'}) \
                                     .find('a') \
                                     .getText()
                vacancy_data['compay_name'] = compay_name
            
                city = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}) \
                              .getText() \
                              .split(', ')[0]
                vacancy_data['city'] = city
            
                metro = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()
                if not metro:
                    metro = None
                else:
                    metro = metro.getText()
                vacancy_data['metro'] = metro
            
                salary = vacancy.find('div', {'class': 'vacancy-serp-item__compensation'})
                if not salary:
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    salary = salary.getText() \
                                   .replace(u'\xa0', u' ')
                
                    salary = re.split(r'\s|-', salary)
                
                    if salary[0] == 'до':
                        salary_min = None
                        salary_max = int(salary[1])
                    elif salary[0] == 'от':
                        salary_min = int(salary[1])
                        salary_max = None
                    else:
                        salary_min = int(salary[0])
                        salary_max = int(salary[1])
                    
                salary_currency = salary[2]
                
                vacancy_data['salary_min'] = salary_min
                vacancy_data['salary_max'] = salary_max
                vacancy_data['salary_currency'] = salary_currency
            
                vacancy_link = vacancy.find('div', {'class' : 'resume-search-item__name'}) \
                                      .find('a'),['href']
                vacancy_data['vacancy_link'] = vacancy_link
                
                vacancies.append(vacancy_data) 
           

 200


In [85]:
pprint(vacancies)

In [81]:
df = pd.DataFrame(vacancy_data)